In [1]:
from datetime import timedelta
from rich import print
from dynaconf import Dynaconf
from arango import ArangoClient
from utinni import Context
from pathlib import Path

from tiro.plugins.utinni import TiroTSPump
from tiro.plugins.arango import ArangoAgent
from tiro.core import Scenario
from influxdb_client import InfluxDBClient

conf = Dynaconf(settings_files=["./config/utinni_config.toml"])
scenario = Scenario.from_yaml(Path("./config/scenario1.yaml"), Path("./config/use1.yaml"))

In [2]:
context = Context(clients=dict(influxdb=InfluxDBClient(**conf.influxdb),
                               arangodb=ArangoAgent(**conf.arangodb)),
                  **conf.on_request)

context.add_pump("tiro", TiroTSPump(scenario=scenario))

In [3]:
table = context.tiro_table(".*Server%(CPU|Memory)Temperature", column="Server", only_ts=False, fill_with_default=True)

In [4]:
context.bind(start=-timedelta(hours=1),
             step=timedelta(minutes=10))

# table.value
# table["MemoryTemperature"].value
table["CPUTemperature"].value

Server,040b2d0c-d675-11ec-b37b-0242ac170002,040b2db6-d675-11ec-b37b-0242ac170002,040b2e2e-d675-11ec-b37b-0242ac170002,040b2e9c-d675-11ec-b37b-0242ac170002,040b2f1e-d675-11ec-b37b-0242ac170002,040b2f8c-d675-11ec-b37b-0242ac170002,040b2ffa-d675-11ec-b37b-0242ac170002,040b3086-d675-11ec-b37b-0242ac170002,040b30f4-d675-11ec-b37b-0242ac170002,040b3180-d675-11ec-b37b-0242ac170002,...,f0f686d4-d855-11ec-a503-0242ac190006,f0f68742-d855-11ec-a503-0242ac190006,f0f687b0-d855-11ec-a503-0242ac190006,f0f68814-d855-11ec-a503-0242ac190006,f0f688e6-d855-11ec-a503-0242ac190006,f0f689b8-d855-11ec-a503-0242ac190006,f0f68a44-d855-11ec-a503-0242ac190006,f0f68aa8-d855-11ec-a503-0242ac190006,f0f68b02-d855-11ec-a503-0242ac190006,f0f68b66-d855-11ec-a503-0242ac190006
2022-05-20 15:20:00+00:00,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0,...,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0
2022-05-20 15:30:00+00:00,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0,...,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0
2022-05-20 15:40:00+00:00,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0,...,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0
2022-05-20 15:50:00+00:00,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0,...,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0
2022-05-20 16:00:00+00:00,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0,...,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0
2022-05-20 16:10:00+00:00,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0,...,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0
2022-05-20 16:20:00+00:00,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0,...,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0


In [5]:
table = context.tiro_table(Path("./config/use1.yaml"), only_ts=False, fill_with_default=True, agg_fn="last")

table.value.keys()

dict_keys(['BackTemperature', 'MemoryTemperature', 'Temperature', 'CPUTemperature', 'Site'])

In [6]:
context.tiro_table(pattern_or_uses=".*Server%(CPU|M|emory)Temperature", type="status", as_df=True, include_tags="Server")["CPUTemperature"].value

,Server,value,unit
0,040b5232-d675-11ec-b37b-0242ac170002,20,°C
1,040b5296-d675-11ec-b37b-0242ac170002,20,°C
2,040b52fa-d675-11ec-b37b-0242ac170002,20,°C
3,040b5390-d675-11ec-b37b-0242ac170002,20,°C
4,040b53f4-d675-11ec-b37b-0242ac170002,20,°C
...,...,...,...
410,f0f689b8-d855-11ec-a503-0242ac190006,20,°C
411,f0f68a44-d855-11ec-a503-0242ac190006,20,°C
412,f0f68aa8-d855-11ec-a503-0242ac190006,20,°C
413,f0f68b02-d855-11ec-a503-0242ac190006,20,°C


In [7]:
print(context.tiro_table(pattern_or_uses=".*Site", type="status", fill_with_default=True).value)

{
    'Room': {
        '040b2974-d675-11ec-b37b-0242ac170002': {
            'Attribute': {
                'Site': {
                    'value': 'Jackson-Castro',
                    'unit': None,
                    'timestamp': '2022-05-18T06:38:10.356888'
                }
            }
        },
        'a398a3e8-d6d6-11ec-8d2a-0242ac170002': {
            'Attribute': {
                'Site': {
                    'value': 'Myers-Joseph',
                    'unit': None,
                    'timestamp': '2022-05-18T18:16:29.892721'
                }
            }
        },
        'f0f6455c-d855-11ec-a503-0242ac190006': {
            'Attribute': {
                'Site': {
                    'value': 'Moore-Long',
                    'unit': None,
                    'timestamp': '2022-05-20T16:00:16.842519'
                }
            }
        }
    }
}

In [8]:
print(context.tiro_table(Path("./config/use2.yaml"), type="status", fill_with_default=True).value)

{
    'Room': {
        '040b2974-d675-11ec-b37b-0242ac170002': {
            'Attribute': {
                'Site': {
                    'value': 'Jackson-Castro',
                    'unit': None,
                    'timestamp': '2022-05-18T06:38:10.356888'
                }
            },
            'Telemetry': {
                'Temperature': {
                    'value': 17.8,
                    'unit': '°C',
                    'timestamp': '2022-05-18T18:05:21.201380'
                }
            }
        },
        'a398a3e8-d6d6-11ec-8d2a-0242ac170002': {
            'Attribute': {
                'Site': {
                    'value': 'Myers-Joseph',
                    'unit': None,
                    'timestamp': '2022-05-18T18:16:29.892721'
                }
            },
            'Telemetry': {
                'Temperature': {
                    'value': 27.76,
                    'unit': '°C',
                    'timestamp': '2022-05-18T18:22:30.241868'
                }
            }
        },
        'f0f6455c-d855-11ec-a503-0242ac190006': {
            'Attribute': {
                'Site': {
                    'value': 'Moore-Long',
                    'unit': None,
                    'timestamp': '2022-05-20T16:00:16.842519'
                }
            },
            'Telemetry': {
                'Temperature': {
                    'value': 25.14,
                    'unit': '°C',
                    'timestamp': '2022-05-20T16:26:18.480863'
                }
            }
        }
    }
}